In [4]:
%load_ext autoreload
%autoreload 2

In [7]:
# from align import eeg2latent
# from maeeg import Encoder
from torch.utils.data import DataLoader
eeg_config= {'preprocessing':{'audio': {'sampling_rate': 16000,
  'max_wav_value': 32768.0,
  'duration': 10,
  'hop_length': 4},
 'stft': {'filter_length': 1024, 'hop_length': 160, 'win_length': 1024},
 'mel': {'n_mel_channels': 64, 'mel_fmin': 0, 'mel_fmax': 8000}},
}
from audioldm_train.utilities.data.eeg_dataset import EEGDataset
from torch.utils.data import Subset
import numpy as np
import time
from tqdm import tqdm

dataset = EEGDataset(eeg_config,split="train")
subsete = Subset(dataset, np.arange(0,len(dataset),2))
subseto = Subset(dataset, np.arange(1,len(dataset),2))
# sample = next(iter(loader))

In [9]:
loader = DataLoader(subsete,batch_size=80,num_workers=4)
for k in tqdm(loader):
    op=0
    #12m
    #7m for subseto from scratch prb 1m for reload

100%|██████████| 705/705 [01:03<00:00, 11.17it/s]


In [14]:
import torch
loader = DataLoader(subsete,batch_size=80,num_workers=4)
with torch.profiler.profile() as prof: 
    for k in tqdm(loader):
        pass  

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


/home/pranav/miniconda3/envs/ae24/lib/python3.10/site-packages/torch/profiler/profiler.py:403: UserWarning: use_cuda is deprecated, use activities argument instead
  warn("use_cuda is deprecated, use activities argument instead")
STAGE:2024-04-15 20:45:54 7154:7154 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
 26%|██▌       | 184/705 [02:17<06:28,  1.34it/s]
STAGE:2024-04-15 20:48:11 7154:7154 ActivityProfilerController.cpp:300] Completed Stage: Collection


KeyboardInterrupt: 

In [13]:
prof.key_averages().table(sort_by="cpu_time_total", row_limit=1)

'-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  \n                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  \n-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  \nenumerate(DataLoader)#_MultiProcessingDataLoaderIter...        99.99%      859.080s       100.00%      859.132s        1.217s           706  \n-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  \nSelf CPU time total: 859.144s\n'

In [39]:
bss=[40,60,100,120,180]
nws=[8,4,2,1]

times2={}
for bs in bss:
    for nw in nws:
        loader = DataLoader(dataset,batch_size=bs,num_workers=nw)
        st_time=time.time()
        for k in tqdm(loader):
            op=0
        times2[(bs,nw)]=time.time()-st_time



100%|██████████| 314/314 [04:29<00:00,  1.16it/s]


In [40]:
times

{(40, 8): 637.7001836299896,
 (40, 4): 65.53898859024048,
 (40, 2): 128.1214940547943,
 (40, 1): 277.3267126083374,
 (60, 8): 35.204561948776245,
 (60, 4): 64.29924726486206,
 (60, 2): 127.0830717086792,
 (60, 1): 274.5298385620117,
 (100, 8): 35.51225757598877,
 (100, 4): 63.270182371139526,
 (100, 2): 128.9572789669037,
 (100, 1): 272.76613545417786,
 (120, 8): 35.15457010269165,
 (120, 4): 63.13059639930725,
 (120, 2): 128.20298194885254,
 (120, 1): 252.97503685951233,
 (180, 8): 35.799540996551514,
 (180, 4): 61.89257621765137,
 (180, 2): 127.11299967765808,
 (180, 1): 269.82686257362366}

In [1]:
import gc

gc.collect() 
gc.get_count()

(62, 0, 0)

In [35]:
#1 min for eeg only 4w
#4min for rand audlat 4w


from tqdm import tqdm
for k in tqdm(loader):
    op=0

 15%|█▍        | 280/1879 [00:40<03:52,  6.87it/s]


KeyboardInterrupt: 

In [27]:
import torch
import numpy as np
import time
from tqdm import tqdm

def timeit(fns,path,iter):
    if "pt" in path:
        tensor_pytorch = torch.randn(512, 768)
        torch.save(tensor_pytorch, path)
        
    elif "npy" in path:
        array_numpy = np.random.randn(512, 768)
        np.save(path, array_numpy)

    load_times=[]
    for _ in range(iter):
        st_time = time.time()
        x=path
        for fn in fns:
            x = fn(x)
        load_times.append(time.time()-st_time)
    
    sum_time= sum(load_times)        
    avg_time = sum_time / len(load_times)
    print(f"Average loading time for {fns}: {avg_time:.6f} seconds , {sum_time:.3f} s")

iters=1000
timeit([np.load],'./delet_me.npy',iters)
timeit([np.load,torch.from_numpy],'./delet_me.npy',iters)
timeit([torch.load],'./delet_me.pt',iters)

Average loading time for [<function load at 0x7f8f4c1f2950>]: 0.001313 seconds , 1.313 s
Average loading time for [<function load at 0x7f8f4c1f2950>, <built-in method from_numpy of type object at 0x7f8f4ee4a460>]: 0.001315 seconds , 1.315 s
Average loading time for [<function load at 0x7f8e54c2db40>]: 0.001480 seconds , 1.480 s


In [24]:
import torch
import numpy as np
import time
from tqdm import tqdm
# File paths
torch_file_path = './delet_me.pt'
numpy_file_path = './delet_me.npy'

# Initialize lists to collect times
torch_load_times = []
numpy_load_times = []
numpy_load_mmap_times = []
numpy_to_tensor_times = []
numpy_to_tensor_times2=[]
# Perform 100 iterations for each process
for _ in tqdm(range(1000)):
    # Create random tensors and arrays
    tensor_pytorch = torch.randn(512, 768)
    array_numpy = np.random.randn(512, 768)

    # Save tensors and arrays
    torch.save(tensor_pytorch, torch_file_path)
    np.save(numpy_file_path, array_numpy)

    # Measure PyTorch tensor loading time
    start_time = time.time()
    tensor_pytorch = torch.load(torch_file_path)
    torch_load_times.append(time.time() - start_time)

    # Measure NumPy array loading time
    start_time = time.time()
    array_numpy = np.load(numpy_file_path)
    numpy_load_times.append(time.time() - start_time)

    # Measure NumPy array loading time
    start_time = time.time()
    x=np.load(numpy_file_path,mmap_mode="r")[:]
    array_numpy = torch.from_numpy(x.copy())
    numpy_load_mmap_times.append(time.time() - start_time)

    # Measure NumPy array loading and conversion to tensor time
    start_time = time.time()
    array_numpy = np.load(numpy_file_path)
    # numpy_load_times2.append(time.time() - start_time)
    # tensor_from_numpy = torch.from_numpy(array_numpy)
    numpy_to_tensor_times.append(time.time() - start_time)

    start_time = time.time()
    tensor_from_numpy = torch.from_numpy(np.load(numpy_file_path))
    numpy_to_tensor_times2.append(time.time() - start_time)



100%|██████████| 1000/1000 [00:58<00:00, 17.09it/s]


In [25]:
# Calculate averages
avg_time_torch = sum(torch_load_times) / len(torch_load_times)
avg_time_numpy = sum(numpy_load_times) / len(numpy_load_times)
avg_time_numpy_mmap = sum(numpy_load_mmap_times) / len(numpy_load_mmap_times)
avg_time_numpy_to_tensor = sum(numpy_to_tensor_times) / len(numpy_to_tensor_times)
avg_time_numpy_to_tensor2 = sum(numpy_to_tensor_times2) / len(numpy_to_tensor_times2)


print(f"Average loading time for PyTorch: {avg_time_torch:.6f} seconds , {sum(torch_load_times):.3f} s")
print(f"Average loading time for NumPy: {avg_time_numpy:.6f} seconds, {sum(numpy_load_times):.3f} s")
print(f"Average loading time for NumPy with mmap: {avg_time_numpy_mmap:.6f} seconds, {sum(numpy_load_mmap_times):.3f} s")
print(f"Average loading time for NumPy with conversion: {avg_time_numpy_to_tensor:.6f} seconds, {sum(numpy_to_tensor_times):.3f} s")
print(f"Average loading time for NumPy with conversion2: {avg_time_numpy_to_tensor2:.6f} seconds, {sum(numpy_to_tensor_times2):.3f} s")

Average loading time for PyTorch: 0.005957 seconds , 5.957 s
Average loading time for NumPy: 0.002071 seconds, 2.071 s
Average loading time for NumPy with mmap: 0.003668 seconds, 3.668 s
Average loading time for NumPy with conversion: 0.002029 seconds, 2.029 s
Average loading time for NumPy with conversion2: 0.001886 seconds, 1.886 s


In [ ]:
from torch import nn
maeeg_config= {

        "params": {
            "img_size": (64, 640),  # eeg x time (10s)
            "in_chans": 1,
            "patch_size": (8, 10),
            "enc_embed_dim": 768,
            "enc_depth": 10,
            "enc_num_heads": 12,
            "dec_embed_dim": 256,  # try 512
            "dec_depth": 6,
            "dec_num_heads": 8,
            "mlp_ratio": 4,
            "norm_layer": nn.LayerNorm,
            "pos_trainable": False,
            "pos_dim": 1,
            "mask_ratio": 0,  # increase compute
            "device":"cpu"
        },
    }
temo=eeg2latent(**maeeg_config['params'])
enc=Encoder(**maeeg_config['params'])

In [ ]:
eeg=sample["eeg"][:,None]
answer=temo.encoder(eeg,0)
answer[0].shape

In [ ]:
enc(eeg,0)[0].shape

In [ ]:
64 * 640/80

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
torch.compile

<function torch.compile(model: Optional[Callable] = None, *, fullgraph: bool = False, dynamic: Optional[bool] = None, backend: Union[str, Callable] = 'inductor', mode: Optional[str] = None, options: Optional[Dict[str, Union[str, int, bool]]] = None, disable: bool = False) -> Callable>